In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy = True

# pyEPR Calculating Disipative Participation Ratios

Zlatko Minev

**Summary:** Following Appendix E of the energy-participation-ratio (EPR) paper, here we demonstrate how to calcualte the dielectric EPR of a chip substrate in a qubit eigen mode. 

We use the following definitions for the RMS energy stored in a volume $V$,
\begin{align}
\mathcal{E}_{\mathrm{elec}}&=&\frac{1}{4}\mathrm{Re}\int_{V}\mathrm{d}v\vec{E}_{\text{max}}^{*}\overleftrightarrow{\epsilon}\vec{E}_{\text{max}}\;,\\\mathcal{E}_{\mathrm{mag}}&=&\frac{1}{4}\mathrm{Re}\int_{V}\mathrm{d}v\vec{H}_{\text{max}}^{*}\overleftrightarrow{\mu}\vec{H}_{\text{max}}\;,
\end{align}

## The simple way  

Following the first tutorial, let's loads the `pyEPR` package under the shorthand name `epr`.

In [2]:
import pyEPR as epr

#### Load Ansys HFSS tutorial file  

As we did in the previous tutorial, let us first determine where the example file is stored.
For tutorial, let us get the path to the tutorial folder.

In [3]:
# Load Path temporarily just to find where the tutorial folder is
# return path_to_project
from pathlib import Path
path_to_project = Path(epr.__file__).parent.parent / '_example_files'
print(f'We will the example project located in\n {path_to_project}')

We will the example project located in
 C:\Miniconda\envs\virenv\lib\site-packages\_example_files


Now, we will open Ansys Desktop and connect to a specific project and we will create the analsys eprh

In [4]:
path_to_project = r'C:\Users\Lehnert Lab\Documents\Sheng-Xiang\pyEPR\_example_files'

pinfo = epr.ProjectInfo(project_path = path_to_project, 
                         project_name = 'pyEPR_tutorial1',
                         design_name  = '1. single_transmon')
eprh = epr.DistributedAnalysis(pinfo)

INFO 01:34PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:34PM [load_ansys_project]: 	File path to HFSS project found.
INFO 01:34PM [load_ansys_project]: 	Opened Ansys App
INFO 01:34PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 01:34PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Lehnert Lab/Documents/Sheng-Xiang/pyEPR/_example_files/
	Project:   pyEPR_tutorial1
INFO 01:34PM [connect_design]: 	Opened active design
	Design:    1. single_transmon [Solution type: Eigenmode]
INFO 01:34PM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:34PM [connect]: 	Connected to project "pyEPR_tutorial1" and design "1. single_transmon" 😀 



Design "1. single_transmon" info:
	# eigenmodes    2
	# variations    6


# Calculate participation of the substrate for mode 1 

First, select which eigenmode to work on. Here the fundamental mode, mode 0, is the qubit.
```python
eprh.set_mode(0) 
```

Let us now calculate the dielectric energy-participatio ratio
of the substrate  relative to the dielectric energy of all objects, using the function
```python 
eprh.calc_p_electric_volume
```
Note that when all objects are specified, this does not include any energy
that might be stored in any lumped elements or lumped capacitors.

Returns:
---------
    ℰ_object/ℰ_total, (ℰ_object, _total)

In [5]:
eprh.set_mode(0) 

# Calculate the EPR p_dielectic
p_dielectic, (ℰ_substr, ℰ_total) = eprh.calc_p_electric_volume('substrate', 'AllObjects')

print(f'Energy in silicon substrate  = {100*p_dielectic:.1f}%')

Energy in silicon substrate  = 87.5%


Now, compute the electric energy stored in the vacuum 
Use the calculated total energy in all objects
so that we don't have to recompute it, since we 
computed it  above

In [6]:
# Here we will pass in the precomputed E_total=ℰ_total
p_vac, (ℰ_vac, ℰ_total) = eprh.calc_p_electric_volume('cavity_enclosure', E_total=ℰ_total)

print(f'''Energy in vacuum = {100*p_vac:.1f}%

Since there are no other volumes,
the two energies should sum to one: {p_dielectic + p_vac}''')

Energy in vacuum = 12.5%

Since there are no other volumes,
the two energies should sum to one: 0.9999999999999983


Let's find outmore about the functuion signature 

In [7]:
? eprh.calc_p_electric_volume

## Calculating the energies directly

Using lower level functions 

In [16]:
ℰ_total  = eprh.calc_energy_electric(obj='AllObjects')
ℰ_substr = eprh.calc_energy_electric(obj='substrate')
print(f'Energy in substrate = {100*ℰ_substr/ℰ_total:.1f}%')

Energy in substrate = 87.7%


In [17]:
?eprh.calc_energy_electric

Signature:
eprh.calc_energy_electric(
    variation=None,
    volume='AllObjects',
    smooth=False,
)
Docstring:
Calculates two times the peak electric energy, or 4 times the RMS, :math:`4*\mathcal{E}_{\mathrm{elec}}`
(since we do not divide by 2 and use the peak phasors).

.. math::
    \mathcal{E}_{\mathrm{elec}}=\frac{1}{4}\mathrm{Re}\int_{V}\mathrm{d}v\vec{E}_{\text{max}}^{*}\overleftrightarrow{\epsilon}\vec{E}_{\text{max}}


volume : string | 'AllObjects'
smooth : bool | False
    Smooth the electric field or not when performing calculation

Example use to calcualte the energy participation of a substrate

.. code-block python
    ℰ_total  = epr_hfss.calc_energy_electric(volume='AllObjects')
    ℰ_substr = epr_hfss.calc_energy_electric(volume='Box1')
    print(f'Energy in substrate = {100*ℰ_substr/ℰ_total:.1f}%')
File:      c:\zkm-code\pyepr\pyepr\core.py
Type:      method


# Using the Fields calculator in HFSS directly 

We will do the same calculation again, but now using the internals of `eprh.calc_energy_electric` to demonstrate how the fields calcualtor object can be used for custom integrals and how the internals work. 

#### Using the HFSS Fields Calculator
The Fields calculator enables you to perform computations using basic field quantities. The calcu­lator will compute derived quantities from the general electric field solution; write field quantities to files, locate maximum and minimum field values, and perform other operations on the field solu­tion. 

The calculator does not perform the computations until a value is needed or is forced for a result. This makes it more efficient, saving computing resources and time; you can do all the calculations without regard to data storage of all the calculated points of the field. It is generally easier to do all the calculations first, then plot the results.

#### Direct calculation of 

\begin{align}
\mathcal{E}_{\mathrm{elec}}&=&\mathrm{Re}\int_{V}\mathrm{d}v\vec{E}_{\text{max}}^{*}\overleftrightarrow{\epsilon}\vec{E}_{\text{max}}\;.
\end{align}



In [8]:
from pyEPR.core import *
from pyEPR.ansys import CalcObject

self, volume = eprh, 'AllObjects'

calcobject = CalcObject([], self.setup)
vecE = calcobject.getQty("E").smooth()
A = vecE.times_eps()
B = vecE.conj()
A = A.dot(B)
A = A.real()
A = A.integrate_vol(name=volume)

E_total = A.evaluate(lv=self._get_lv()) 
# This command numerically evaluates and displays the
# results of calculator operations 
E_total

4.82017880409214e-22

In [9]:
from pyEPR.core import *

self, volume = eprh, 'substrate'

calcobject = CalcObject([], self.setup)
vecE = calcobject.getQty("E").smooth()
A = vecE.times_eps()
B = vecE.conj()
A = A.dot(B)
A = A.real()
A = A.integrate_vol(name=volume)

E_subs = A.evaluate(lv=self._get_lv()) 
# This command numerically evaluates and displays the
# results of calculator operations 
E_subs

4.25224091862523e-22

In [10]:
print(f'Energy in substrate: {100*E_subs/E_total:.1f}%')

Energy in substrate: 88.2%


# Summary  

We showed three levels of how to obtain the same result, using either the 
1. `pyeprh.calc_p_electric_volume`
2. `pyeprh.calc_energy_electric`
3. Or the fields calcualtor

**NEXT:** Please see *Part III* of the tutorial to continue. You will find it in the tutorial folder as a jupyter notebook. 